In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install seaborn
!pip install evaluate
!pip install sklearn
!pip install sentencepiece
!pip install accelerate -U
!pip install kornia

In [5]:
from transformers import AutoModel,BertTokenizerFast,AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
import re
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm import tqdm_notebook
from datasets import load_dataset, load_metric, Dataset
import pandas as pd
from src.loss_function import *

In [6]:
list_of_models  =["UBC-NLP/MARBERT", "qarib/bert-base-qarib", "aubmindlab/bert-base-arabertv02-twitter",'aubmindlab/bert-large-arabertv02']

model_name=list_of_models[0]

# model_checkpoint = model_name.split("/")[-1]+"_"+"focal_loss"
model_checkpoint = model_name.split("/")[-1]

batch_size = 16

learning_rate = 0.00002

epochs = 4


In [7]:
acc_metric = load_metric('accuracy')
f1_metric = load_metric('f1')
precision_metric = load_metric('precision')
recall_metric = load_metric('recall')

<ipython-input-7-cba645162e86>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  acc_metric = load_metric('accuracy')


In [8]:
def compute_metrics(eval_pred):

  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)

  accuracy = acc_metric.compute(predictions=predictions, references=labels)
  f1 = f1_metric.compute(predictions=predictions, references=labels,average='macro')
  precision = precision_metric.compute(predictions=predictions, references=labels,average='macro')
  recall = recall_metric.compute(predictions=predictions, references=labels,average='macro')

  return {"accuracy":accuracy['accuracy'],"f1":f1['f1'],"precision":precision['precision'],"recall":recall['recall']}

In [9]:
def upload_dataset(trian_file,valid_file,test_file):

  dataset = load_dataset("csv", data_files=trian_file)
  val_data = pd.read_csv(valid_file)
  ds_val = Dataset.from_pandas(val_data)

  test_data = pd.read_csv(test_file)
  ds_test = Dataset.from_pandas(test_data)

  dataset["validation"] = ds_val
  dataset["test"] = ds_test

  return dataset


In [11]:
# MAX_LEN = 96   #MARBERT
# MAX_LEN = 60    #qarib/bert-base-qarib
# MAX_LEN = 52    #aubmindlab/bert-base-arabertv02-twitter / aubmindlab/bert-large-arabertv02



# MAX_LEN = 329   #MARBERT XLNI
# MAX_LEN = 359   #qarib/bert-base-qarib
# MAX_LEN = 329    #aubmindlab/bert-base-arabertv02-twitter / aubmindlab/bert-large-arabertv02 X_NLI


MAX_LEN = 90   #MARBERT ArbTEDS
# MAX_LEN = 97   #qarib/bert-base-qarib
# MAX_LEN = 88    #aubmindlab/bert-base-arabertv02-twitter / aubmindlab/bert-large-arabertv02 X_NLI



def preprocess_function(examples):
    return tokenizer(examples['t'], examples['h'], truncation=True, padding="max_length", max_length=MAX_LEN)





tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

dataset = upload_dataset('dataset/ArbTEDS/train_ArbTEDS.csv','dataset/ArbTEDS/valid_ArbTEDS.csv','dataset/ArbTEDS/test_ArbTEDS.csv')

encoded_dataset = dataset.map(preprocess_function)


Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [12]:
args = TrainingArguments(
    f"{model_checkpoint}_checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    load_best_model_at_end=True,

)


In [13]:
from torch import nn
from transformers import Trainer

# This block only for usng focal loss function
# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         # forward pass
#         outputs = model(**inputs)
#         logits = outputs.get("logits")
#         loss_fct = FocalLoss(alpha=0.5, gamma=2.0, reduction='mean')
#         loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
#         return (loss, outputs) if return_outputs else loss

In [14]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<IPython.core.display.Javascript object>

In [ ]:
predictions = trainer.predict(encoded_dataset["test"])



/localscratch/omar8.5845327.0/jupyter_new/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [ ]:
predictions